# We're scraping

Probably going to just scrape it all. 

See [baseball-scraper](https://pypi.org/project/baseball-scraper/) for examples.

In [2]:
from baseball_scraper import statcast
from baseball_scraper import playerid_lookup

import pandas as pd 
import datetime 

In [2]:
##What happens if I ask for a date on which no games are played... 
##Like I write a function and uh-oh, all-star break included...

##Looking for a date pre statcast gives a table with lots of NaN values. 
#pre_stat_scrape = statcast(start_dt='2010-08-08', verbose=True)

##Looking for a date on which no games were played gives a dataframe of shape 0x90
#january_scrape = statcast(start_dt='2018-01-01', verbose=True)




'2018-01-01'

In [4]:
start = datetime.date(2020,7,23).strftime("%Y-%m-%d")
end = datetime.date(2020,8,18).strftime("%Y-%m-%d")##This is the world's third hardest way to write '2020-08-18', but you're maybe using it later.
start

'2020-07-23'

In [12]:
## Scrapes time slices based on a date range. Returns a list of successfully scraped time slices.

def big_scrape(start,end):
    '''
    Returns a DataFrame of all the successfull scraped time slices in range start, end. 
    
    If an exception     is raised, prints a brief report, stops scraping, and returns
    all the successfully scraped slices.
    '''
    scrapes = [] ##
    while True:
        try:
            temp = statcast(start.strftime("%Y-%m-%d"), (start+datetime.timedelta(days=4)).strftime("%Y-%m-%d"))
        except ParserError:
            print("ParserError failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        except:
            print("Unspecified failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        scrapes.append(temp)
        start = start+datetime.timedelta(days=5)
        if start > end:
            break
    return pd.concat(scrapes)

In [9]:
## Creates all the .csv files.

for dt in [2019,2018,2017,2016]:
    start = datetime.date(dt,3,20)
    end = datetime.date(dt,10,1)
    scrape = big_scrape(start,end)
    scrape.to_csv('data/statcast_dumps/statcast'+str(dt)+'.csv')
    print("Finished with statcast year {}.".format(dt))
    del scrape

Finished with statcast year 2019.
Finished with statcast year 2018.
Finished with statcast year 2017.
Finished with statcast year 2016.


In [20]:
## updating the current year efficiently

def update_statcast(target_csv_name):
    '''
    Scapes any statcast data after the last game_date 
    in the year of the target file and adds it, returning a DataFrame that concatenates the
    new data with the old.
    
    '''
    out = pd.read_csv(target_csv_name)
    start = pd.Timestamp(out['game_date'].sort_values().iloc[-1])
    end = datetime.datetime.now()
    new = pd.concat(big_scrape(start, end))
    out = pd.concat([out,new])
    return out




In [21]:
sc_2020 = update_statcast('data/statcast_dumps/statcast2020.csv')

2020-08-18 00:00:00 2020-08-21 08:13:17.989416


In [22]:
sc_2020

,Unnamed: 0,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0.0,25,SL,2020-08-18,85.7,-0.753952,5.837731,Anthony Bass,592407.0,542914.0,...,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Infield shift,Standard
1,1.0,31,SL,2020-08-18,85.9,-0.705461,5.807248,Anthony Bass,592407.0,542914.0,...,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Infield shift,Standard
2,2.0,34,SI,2020-08-18,95.8,-0.704409,5.935278,Anthony Bass,592407.0,542914.0,...,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Infield shift,Standard
3,3.0,39,SI,2020-08-18,95.3,-0.397764,5.941654,Anthony Bass,592407.0,542914.0,...,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Infield shift,Standard
4,4.0,41,SI,2020-08-18,94.3,-0.287143,6.163535,Anthony Bass,592407.0,542914.0,...,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Infield shift,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13130,NaN,13078,FF,2020-08-18 00:00:00,97.7,-1.301370,5.691097,Luke Weaver,543760.0,596133.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infield shift,Standard
13131,NaN,13079,FC,2020-08-18 00:00:00,91.8,-1.192651,5.699736,Luke Weaver,543760.0,596133.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infield shift,Standard
13132,NaN,13098,CH,2020-08-18 00:00:00,85.8,-1.455621,5.622799,Luke Weaver,543760.0,596133.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infield shift,Standard
13133,NaN,13119,FF,2020-08-18 00:00:00,95.2,-1.335501,5.584656,Luke Weaver,543760.0,596133.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infield shift,Standard
